## Exercise  7  |  data  in  f2ex7  |  For  each  “Freguesia”  in  Coimbra  (freg_cmb.shp),  provide  the average distance (in kilometers) to the nearest school (cmb_escolas.shp). 

In [ ]:
units = '/mnt/d/cwsig/exercicios_2023_24/f2ex7/freg_cmb.shp'

pois = '/mnt/d/cwsig/exercicios_2023_24/f2ex7/cmb_escolas.shp'

outunits = '/mnt/d/cwsig/exercicios_2023_24/f2ex7/freg_cmb_res.shp'

In [ ]:
import os

from glass.wt.rst import shpext_to_rst
from glass.pys.oss import mkdir, fprop
from glass.pys.tm import now_as_str
from glass.wenv.grs import run_grass

In [ ]:
ws, loc = mkdir(os.path.join(
    os.path.dirname(outunits),
    now_as_str(utc=True)
), overwrite=True), 'loc_unitmean'

refrst = shpext_to_rst(units, os.path.join(ws, 'refrst.tif'), cellsize=10)

gb = run_grass(ws, location=loc, srs=refrst)

# Start grass gis session
import grass.script.setup as gsetup
gsetup.init(gb, ws, loc, 'PERMANENT')

In [ ]:
from glass.it.shp import shp_to_grs, grs_to_shp
from glass.it.rst import grs_to_rst
from glass.rst.dst import grow_distance
from glass.dtt.torst import grsshp_to_grsrst
from glass.rst.zon.grs import grs_rst_stats_by_feat
from glass.tbl.col import rn_cols

In [ ]:
# Import data into GRASS GIS
grs_units = shp_to_grs(units)
grs_pois = shp_to_grs(pois)

# POIS to Raster
rst_pois = grsshp_to_grsrst(grs_pois, 1, f"rst_{grs_pois}")

# Calculate Euclidean distance to POIS
gist = grow_distance(rst_pois, 'dpois', api="grass")

# Get average for each Unit
grs_rst_stats_by_feat(grs_units, gist, 'dpois', 'average')
rn_cols(grs_units, {'dpois_average': 'dpois'}, api='grass')

# Export results
grs_to_rst(gist, os.path.join(ws, f'{gist}.tif'), rtype=float)

grs_to_shp(grs_units, outunits, 'area')